In [1]:
%load_ext autoreload
%autoreload 2

%connect_info

{
  "shell_port": 37161,
  "iopub_port": 49271,
  "stdin_port": 53777,
  "control_port": 39601,
  "hb_port": 59777,
  "ip": "127.0.0.1",
  "key": "490dc018-8d0c705ce85cf841c2747cf6",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-77351a90-dab3-41b9-94e9-5398e31c2081.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
import django

In [3]:
django.setup()

In [4]:
"""
Project Configuration Sheets
"""

FIN_MAN_PROJECT_SHEET_NAME = "Finman Project Configuration"
FIN_MAN_CONFIGURATION_SHEET_NAME="FinMan Configuration_v4 (Payment Request Complete Config)"
VENDOR_CONFIGURATION_SHEET_NAME = "Vendor Configuration_v1 - Test"

In [5]:
"""
Populate project and project roles
"""

from django.db import transaction

@transaction.atomic()
def populate_project_and_roles():
    from ib_iam.populate.populate_utils import populate_projects
    populate_projects(spread_sheet_name=FIN_MAN_PROJECT_SHEET_NAME)

    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(FIN_MAN_CONFIGURATION_SHEET_NAME)

In [6]:
"""
Populate tasks related data
"""

from django.db import transaction

@transaction.atomic()
def populate_task_and_boards_related_data():
    
    configuration_sheets = [
        FIN_MAN_CONFIGURATION_SHEET_NAME,
        VENDOR_CONFIGURATION_SHEET_NAME
    ]

    for configuration_sheet in configuration_sheets:

        from ib_tasks.populate.population_utils import populate_data
        populate_data(configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

In [7]:
"""
Populate project related data
"""
from django.db import transaction

@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(FIN_MAN_PROJECT_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(FIN_MAN_PROJECT_SHEET_NAME)

In [8]:
populate_project_and_roles()
populate_task_and_boards_related_data()
populate_project_related_data()

[BoardDTO(board_id='FINB_PAYMENT_REQUESTS', name='Payment Requests'), BoardDTO(board_id='FINB_ACCOUNTS_L1_VERIFICATION_PRE_PAYMENT', name='Pre Payment Accounting Verification'), BoardDTO(board_id='FINB_ACCOUNTS_L2_VERIFICATION_PRE_PAYMENT', name='Pre Payment Accounting Verification'), BoardDTO(board_id='FINB_ACCOUNTS_L3_VERIFICATION_PRE_PAYMENT', name='Pre Payment Accounting Verification'), BoardDTO(board_id='FINB_ACCOUNTS_L4_VERIFICATION_PRE_PAYMENT', name='Pre Payment Accounting Verification'), BoardDTO(board_id='FINB_ACCOUNTS_VERIFICATION_PRE_PAYMENT', name='Pre Payment Accounting Verification'), BoardDTO(board_id='FINB_PAYMENTS_L1_VERIFICATION_PRE_PAYMENT', name='Payment Requests verification'), BoardDTO(board_id='FINB_PAYMENTS_L2_VERIFICATION_PRE_PAYMENT', name='Payment Requests verification'), BoardDTO(board_id='FINB_PAYMENTS_L3_VERIFICATION_PRE_PAYMENT', name='Payment Requests verification'), BoardDTO(board_id='FINB_PAYMENTS_L5_VERIFICATION_PRE_PAYMENT', name='Payment Requests v

[BoardDTO(board_id='FINB_VENDOR_REGISTRATION', name='Vendors Added by Me'), BoardDTO(board_id='FINB_VENDOR_APPROVAL', name='Vendors RP Approval'), BoardDTO(board_id='FINB_PV5_VENDOR_VERIFICATION', name='Vendors P5 Approval'), BoardDTO(board_id='FINB_PV4_VENDOR_VERIFICATION', name='Vendors P4 Approval'), BoardDTO(board_id='FINB_AV4_VENDOR_VERIFICATION', name='Vendors A5 Approval')] [ColumnDTO(column_id='FINC_COLUMN_DRAFTS', name='Drafts', display_order=1, task_template_stages=[TaskTemplateStagesDTO(task_template_id='FIN_VENDOR', stages=['VENDOR_DRAFTS'])], user_role_ids=['FIN_PAYMENT_REQUESTER'], column_summary='ColumnSummary1', column_actions='ColumnSummary3', list_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], kanban_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], board_id='FINB_VENDOR_REGISTRATION'), ColumnDTO(column_id='FINC_PENDING_RP_APPROVAL', name='RP Approval', display_order=2, task_template_stages=[TaskTemplateStagesDTO(ta

In [9]:
from ib_tasks.populate.write_stage_actions import writing_data_to_stage_actions_logic
writing_data_to_stage_actions_logic()

[none] [ib_workflows_backend - local] INFO     [13:43:27]  [/usr/lib/python3.8/lib2to3/pgen2/driver.py] [driver.py][load_grammar] [120]: Generating grammar tables from /usr/lib/python3.8/lib2to3/Grammar.txt
[none] [ib_workflows_backend - local] INFO     [13:43:27]  [/usr/lib/python3.8/lib2to3/pgen2/driver.py] [driver.py][load_grammar] [120]: Generating grammar tables from /usr/lib/python3.8/lib2to3/PatternGrammar.txt


In [11]:
"""
Populate Ib IAM Related Data For FinMan Project Configuration

Note:- While Running the this below script need to Run the ELASTIC SEARCH SERVER
"""
from ib_iam.populate.populate_utils import populate
populate(FIN_MAN_CONFIGURATION_SHEET_NAME)

from ib_users.models import *
UserAccount.objects.create_superuser(username='admin', password='admin@123', email='admin@example.com')
UserAccount.objects.create_superuser(username='fduser', password='fduser@123', email='fduser@example.com')

[none] [ib_workflows_backend - local] INFO     [13:43:51]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/user-local/_doc [status:201 request:0.020s]
[none] [ib_workflows_backend - local] INFO     [13:43:51]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/user-local/_doc [status:201 request:0.011s]
[none] [ib_workflows_backend - local] INFO     [13:43:51]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/user-local/_doc [status:201 request:0.011s]
[none] [ib_workflows_backend - local] INFO     [13:43:51]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/s

<UserAccount: c399982f-4733-4656-bd37-538bb4352dc3>

In [12]:
"""
Populate Elastic Search Data For Cities

"""
from ib_iam.populate.populate_elastic_search_data import populate_data
populate_data()

[none] [ib_workflows_backend - local] INFO     [13:43:56]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/country-local/_doc [status:201 request:0.012s]
[none] [ib_workflows_backend - local] INFO     [13:43:56]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/country-local/_doc [status:201 request:0.012s]
[none] [ib_workflows_backend - local] INFO     [13:43:56]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/country-local/_doc [status:201 request:0.012s]
[none] [ib_workflows_backend - local] INFO     [13:43:56]  [/home/ib-developer/Desktop/work/ib-workflows-backend/venv/lib/py